# GAN Help/Hints
## Tasks

## Have we used the Framework?
Check this link: <https://medium.com/@devnag/generative-adversarial-networks-gans-in-50-lines-of-code-pytorch-e81b79659e3f#.sch4xgsa9>
        

**Creaing a Generator**


###  0. Generator
* Write a subclass that defines the generator:
    `class Generator(nn.Module):`
* Define the `__init__` construct with these parameters:
`(self, input_size, hidden_size, output_size)`
* Make sure you define the feed-forward function inside of the class
    `def forward(self, x):`
* The network should have three layers and two `tanh` activation functions after the first and second layer.
* The layers and activation functions should be contained inside of a `nn.Sequential` wrapper class.


```
student@holbie-ml:~/pytorch$ cat << EOF > ./0-main.py 
#!/usr/bin/env python3


import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
genTester = __import__('0-generator').Generator

G_Test = genTester(1,1,1)

print(G_Test)

EOF

student@holbie-ml:~/pytorch$ python3 ./0-main.py 
Generator(
  (main): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1, out_features=1, bias=True)
    (3): Tanh()
    (4): Linear(in_features=1, out_features=1, bias=True)
  )
)

```

In [9]:
#!/usr/bin/env python3

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable


class Generator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Generator2, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size)
            )
        

    def forward(self, x):
        return self.main(x)


## Changes
### Task 2
    Task 2 is a little unclear. Instead of creating a single function, we will create two tasks out of this one. We will have it be two different functions entirely. Or We can actually just take the task out and combine them inside of the trainting task. 

The mean and standard deviation can be global variables to assist here. 

Mean = 0
StdDev = 1

### Sampler for Generator
```
aTorchTensorObjectToTrainWith = torch.rand(m, n)
```
### 'm' should be the batch size, 'n' should be the input size

### Sampler for Discriminator
```
anotherTorchTensorObjectToTrainWith = torch.Tensor(np.random.normal(mu, sigma, (1, n)))
```
### 'mu' should be the mean, 'sigma' should be standard deviation, 'n' should be the input size

### Train Generator Examples with sampling

Here is an example of the Train Generator object with sampling happening insisde of it:



In [20]:
def train_gen(Gen,Dis, gInputSize, mbatchSize, steps, optimizer, crit):
                
        for g_index in range(steps):
            Gen.zero_grad(set_to_none=False)
        
            gen_input = torch.rand(mbatchSize, gInputSize)
            g_fake_data = Gen(gen_input)
            dg_fake_decision = Dis(processMoments(g_fake_data.t()))
            g_error = crit(dg_fake_decision, Variable(torch.ones([1]).to(mainComputeDevice)))  
            
            g_error.backward()
            optimizer.step()  
            return g_error,g_fake_data

### Getting moments:

Just remember the moments are:
* mean
* variance
* skewness
* kurtosis



In [19]:
def processMoments(d):
    # Return the first 4 moments of the data provided
    mean = torch.mean(d)
    diffs = d - mean
    var = torch.mean(torch.pow(diffs, 2.0))
    std = torch.pow(var, 0.5)
    zscores = diffs / std
    skews = torch.mean(torch.pow(zscores, 3.0))
    kurtoses = torch.mean(torch.pow(zscores, 4.0)) - 3.0  # excess kurtosis, should be 0 for Gaussian
    final = torch.cat((mean.reshape(1,), std.reshape(1,), skews.reshape(1,), kurtoses.reshape(1,)))
    return final

### Train Gan example below


In [18]:
def train_gan():

    g_input_size = 1   # Random noise dimension coming into generator, per output vector
    g_hidden_size = 5  # Generator complexity
    g_output_size = 1  # Size of generated output vector
    minibatch_size =  500 # Minibatch size - cardinality of distributions
    d_input_size = 4 #descriminator input size
    d_hidden_size = 10 # Discriminator complexity
    d_output_size = 1  # Single dimension for 'real' vs. 'fake' classification
    num_epochs = 50
    d_steps = 20
    g_steps = 20

    #learning rate and momentum
    learning_rate = 1e-3
    sgd_momentum = 0.9
    dfe, dre, ge = 0, 0, 0
    d_sub_data, d_real_data, d_fake_data, g_fake_data = None, None, None, None


    G = Generator2(input_size=g_input_size,
                  hidden_size=g_hidden_size,
                  output_size=g_output_size)
    D = Discriminator2(input_size=d_input_size,
                      hidden_size=d_hidden_size,
                      output_size=d_output_size)
                      
    criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
    
    d_optimizer = optim.SGD(D.parameters(), lr=learning_rate, momentum=sgd_momentum)
    g_optimizer = optim.SGD(G.parameters(), lr=learning_rate, momentum=sgd_momentum)

    #place it to a gpu hopefully
    G.to(mainComputeDevice)
    D.to(mainComputeDevice)

    for epoch in range(num_epochs):

        dre, dfe, d_fake_data, d_real_data = train_dis(G,D,d_input_size,g_input_size, minibatch_size, d_steps, d_optimizer, criterion)
        ge, g_fake_data = train_gen(G,D,g_input_size, minibatch_size, g_steps, g_optimizer, criterion)
    return g_fake_data

#fakeData = train_gan()